# 1.2 Relativistic equations of motion

Watch the video https://www.youtube.com/watch?v=InD54gt1P7M&index=2&list=PLdCdV2GBGyXNSUU4-B4I1QtbF6af6pTIr&t=0s

Develop a simulation in the cell below showing a two space craft traveling with a constant acceleration toward alpha-centauri: one subject to classical physics and the other using the relativistic Euler-Cromer method
Alpha centauri is 4.367 light years away from us.

## Notes on the transition from Glowscript to Jupyter
How GlowScript and Jupyter Differ  from Classic VPython

* Vectors must be represented as vector(x,y,z) or vec(x,y,z), not as (x,y,z).

* The name display has been changed to canvas.

* The name gdisplay has been changed to graph.

## Important computational/theoretical skill: scaling units

This is what is meant by "use units where c=1".

SI units would be very inconvenient here. What is more convenient is scaling to dimensionless quantities, and this is an important skill to develop, for computation or theory. Let's start with the momentum update equation. It is already relativistically correct.
$$\vec{p}_f=\vec{p}_i+\vec{F}_{net}\Delta t$$ can be written as
$$\vec{p}_f c=\vec{p}_i c+\vec{F}_{net}c\Delta t$$

Dividing both sides by rest energy we obtain
$$\frac{\vec{p}_f c}{mc^2}=\frac{\vec{p}_i c}{mc^2}+\frac{\vec{F}_{net}}{mc^2}c\Delta t$$

Let us define a dimensionless unit of "momentum" $$\overline{p}\equiv \frac{\vec{p} c}{mc^2}$$ and $$\overline{t}\equiv \frac{ct}{1\,\text{light year}}$$, then it is natural to define the dimensionless ratio $$\overline{F}_{net}\equiv \frac{\vec{F}_{net}\times 1\,\text{light year}}{mc^2}$$ (in other words, the amount of work that would be done by a force over a distance of 1 light year, as a percentage of the rest mass). Thus we obtain the dimensionless update equation
$$\overline{p}_f=\overline{p}_i+\overline{F}_{net} \Delta \overline{t}$$
where \overline{t} is the ratio of time to 1 year (unitless, but you can think of it as "years").

For the position update equation, we have
$$\vec{r}_f = \vec{r}_i + \frac{\vec{p}_f}{m} \Delta t$$

Let's write this in a more convenient units

$$\vec{r}_f = \vec{r}_i + \frac{\vec{p}_fc}{mc^2} c\Delta t$$

Now this isn't quite right, as $\vec{p}=\gamma m\vec{v}$. As worked out on page 210 in Chabay and Sherwood, we can solve for $\vec{v}$:
$$\frac{\vec{v}}{c}=\frac{\vec{p}c}{mc^2}\times \left(1+\frac{pc}{mc^2}\right)^{1/2}$$

Thus we need to multiply by a dimensionless "relativistic factor" to obtain the proper update equation

$$\vec{r}_f = \vec{r}_i + \frac{\vec{p}_fc}{mc^2} [\text{factor}] \Delta t$$

We can introduce entirely dimensionless quantities by dividing both sides by 1 light year and re-defining "length" in these dimensionless units $$\overline{r}\equiv \frac{\vec{r}}{1\,\text{light year}}$$, $$\overline{t} \equiv \frac{ct}{1\,\text{light year}}$$. So $c\overline{t}=1$ is interpreted as the ratio of distance to 1 light year is 1. This is what is meant by the line c=1, and people often will say "set c=1, let distance be in light years, let momentum be  ...well something like that", but this is the more careful way to proceed.


$$\overline{r}_f = \overline{r}_i + \overline{p}_f [\text{factor}] \Delta \overline{t}$$

## LO 1: implement relativistic and non-relativistic momentum principle and position update equation

Complete the code below to show the difference in motion governed by relativity 
The momentum principle is correct, even including the relativistically-correct momentum principle, and the approximate form appropriate for slower speeds. 

### Notes on applying what we already know to relativistic speeds
$$\vec{F}_{net}=\frac{d\vec{p}}{dt}$$
provided we use the correct definition of momentum
$$\vec{p}=\frac{1}{\sqrt{1-v^2/c^2}}m\vec{v}="\gamma" m\vec{v}$$. As explained on page 79 of Chabay and Sherwood, we can solve for $v$
$$\vec{v}=\frac{1}{\sqrt{1+\left(\frac{p_x c}{mc^2}\right)^2}}\frac{p_x}{m}.$$

This looks a lot like $v\approx p/m$ with an additional ``relativistic factor''
$$\vec{v}=\text{``relativistic factor''}\times \frac{\vec{p}}{m},$$ where

$$\text{``relativistic factor''}=\frac{1}{\sqrt{1+\left(\frac{p_x c}{mc^2}\right)^2}}.$$ 
Other than adding this relativistic factor, we can proceed using the Euler-Cromer integration method as usual.



In [1]:
from vpython import *
canvas()
c = 1 #see notes on scaling

graph(xtitle="time", ytitle="position")
pos_graph = gcurve(color=color.red,label="rel x")
nonrel_pos_graph = gcurve(color=color.green,label="nonrel x")
graph(xtitle="time", ytitle="speed")
vel_graph = gcurve(color=color.red,label="rel v_x")
nonrel_vel_graph = gcurve(color=color.green,label="nonrel v_x")

ModuleNotFoundError: No module named 'vpython'

I have had trouble when executing these instructions more than once. That is why I split the code into two parts--so we don't make new graphs every time we execute. You may need to select Kernel>Restart and Run All if you have problems.

In [2]:
destinationDistance = 4.367 # alpha centauri is 4.367 light years away
#We need to make things large enough to see, so multiply sizes by scaleDistance
scaleDistance = 1e-2*destinationDistance

#I'm just looking for an easy way to center the scene
earth = sphere(radius=2*scaleDistance, pos=vector(-destinationDistance/2,0,0), color=color.blue)
alphaCentauri = sphere(radius=2*scaleDistance, pos=vector(destinationDistance/2,0,0), color=color.yellow)

#relativistic craft
rel = sphere( radius=0.5*scaleDistance, pos=earth.pos, color=color.red)
rel.momentum = vector(0,0,0)
rel.mass = 1.0

# non-relativistic craft
nonrel = sphere( radius=0.5*scaleDistance, pos=earth.pos, color=color.green)
nonrel.momentum = vector(0,0,0)
nonrel.mass = 1.0


dt = 0.01
time = 0
while time < 5:
    rate(50)
    force = vector(0.5,0,0)

    # Relativistic ECM.
    rel.momentum = rel.momentum + force*dt
    rel.rel_fac = 1.0 / sqrt(1.0+(mag(rel.momentum)**2/(rel.mass**2*c**2)))
    rel.pos = rel.pos + rel.rel_fac*rel.momentum/rel.mass*dt
    
    # Add non-relativistic ECM here.
    nonrel.momentum = nonrel.momentum + force*dt
    nonrel.pos = nonrel.pos + nonrel.momentum/nonrel.mass*dt
    
    time = time + dt
    pos_graph.plot(pos=(time,rel.pos.x))
    nonrel_pos_graph.plot(pos=(time,nonrel.pos.x))
    velocity = rel.momentum/rel.mass*rel.rel_fac
    vel_graph.plot(pos=(time,velocity.x))
    velocity = nonrel.momentum/nonrel.mass*nonrel.rel_fac
    nonrel_vel_graph.plot(pos=(time,velocity.x))
  

NameError: name 'sphere' is not defined

## LO 2: Are we there yet?
Discuss the following questions
1. How much longer will it take to get to alpha centauri, compared to the incorrect classical result? Estimate the answer to the nearest 10th of a year by looking at the graphs.
2. Does the discrepancy get larger or smaller as the force increases? Explain

In [ ]:
#
#The vpython module wouldn't load in Jupyter, so I used Glowscript:
#https://www.glowscript.org/#/user/Waluigi/folder/Public/program/modernphysics1/edit
#
#1)    Nonrelativistic (classically calulated) time = 4.18 lightyears. Relativistic time = 6.04 lightyears.
#      The Classical value is 1.86 lightyears less.
#
#2)    Larger; if we increase the force to 50000 netwons, classical model shows the object reaching Alpha Centauri almost instantly,
#      while the relativistic model shows it getting there no faster than the speed of light.